In [1]:
# Import dependencies
import pandas as pd
import matplotlib as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key
from datetime import datetime

In [2]:
# 3. Create a set of 2000 random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
    # Putting into zip
lat_lngs = zip(lats, lngs)

    # Unzip into list
coordinates = list(lat_lngs)

In [4]:
# 4. Get the nearest city using citipy
    # Create list to hold cities
cities = []
    # Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # Only add unique cities
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient city count.
len(cities)

758

In [5]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    url = "http://api.openweathermap.org/data/2.5/weather?units=imperial&APPID=" + weather_api_key
    city_url = url + "&q=" + city.replace(" ","+")

    # Log thr url, record, and set numbers and city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record
    record_count+= 1
    # 5. Run an API call for each of the cities for open weather
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # 6. Retrieve needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
       
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

# 7. Conver data to DF and check it
city_data_df = pd.DataFrame(city_data)

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | punta arenas
Processing Record 2 of Set 1 | chokurdakh
Processing Record 3 of Set 1 | saskylakh
Processing Record 4 of Set 1 | vestmannaeyjar
Processing Record 5 of Set 1 | busselton
Processing Record 6 of Set 1 | dikson
Processing Record 7 of Set 1 | barrow
Processing Record 8 of Set 1 | kapaa
Processing Record 9 of Set 1 | jamestown
Processing Record 10 of Set 1 | arraial do cabo
Processing Record 11 of Set 1 | tumannyy
City not found. Skipping...
Processing Record 12 of Set 1 | duku
Processing Record 13 of Set 1 | burica
City not found. Skipping...
Processing Record 14 of Set 1 | oussouye
Processing Record 15 of Set 1 | urucara
Processing Record 16 of Set 1 | necochea
Processing Record 17 of Set 1 | castro
Processing Record 18 of Set 1 | husavik
Processing Record 19 of Set 1 | hermanus
Processing Record 20 of Set 1 | tiksi
Processing Record 21 of Set 1 | saint-philippe
Processing Record 22 of 

In [14]:
# 7. Conver data to DF and check it
city_data_df = pd.DataFrame(city_data)
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df[new_column_order].head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Punta Arenas,CL,-53.1500,-70.9167,37.51,87,20,12.66,few clouds
1,Chokurdakh,RU,70.6333,147.9167,34.74,87,100,16.96,overcast clouds
2,Saskylakh,RU,71.9167,114.0833,33.01,84,100,14.99,overcast clouds
3,Vestmannaeyjar,IS,63.4427,-20.2734,42.80,79,92,6.64,overcast clouds
4,Busselton,AU,-33.6500,115.3333,63.05,55,3,16.46,clear sky
5,Dikson,RU,73.5069,80.5464,35.96,88,100,17.38,overcast clouds
6,Barrow,US,71.2906,-156.7887,28.42,86,90,4.61,light snow
7,Kapaa,US,22.0752,-159.3190,82.38,76,90,1.99,light rain
8,Jamestown,US,42.0970,-79.2353,58.71,87,1,1.01,clear sky
9,Arraial Do Cabo,BR,-22.9661,-42.0278,69.21,77,51,24.02,broken clouds


In [16]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# 8. Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")